# Kernel Mapping Problem (MRRG)

## Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import pillars.core._
import pillars.mapping.thirdParty._
import pillars.archlib.TileLSUBlock
import pillars.hardware.{TopModule}
import pillars.archlib._
import pillars.core.{BlockTrait, OpEnum}
import pillars.core.OpEnum.OpEnum
import scala.collection.mutable.Map

import pillars.mapping.{DFG, DotReader, ILPMap, OmtMap, SearchMap}
import java.util.Date

## Create a simple block

In [ ]:
class SimpleBlockWithReg(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)

    // A register file with 2 registers 
    val rf0 = new ElementRF("rf0", List(1, 1, 1, 32))
    //port sequnces outs: 0: out_0
    //port sequnces inputs: 0: input_0
    rf0.addOutPorts(Array("out_0"))
    rf0.addInPorts(Array("input_0"))
    addElement(rf0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> rf0 / "input_0")
    addConnect(rf0 / "out_0" -> term("out_0"))
}


## Create a very simple DFG

In [ ]:
//result = a + b
def initAdd(g: Digraph): Unit = {
    //Initialize nodes in this DFG
    //the value of a
    g.node("input0", attrs = Map("opcode" -> "input"))
    //the value of b
    g.node("input1", attrs = Map("opcode" -> "input"))
    //a + b
    g.node("add0", attrs = Map("opcode" -> "add"))
    //result
    g.node("output0", attrs = Map("opcode" -> "output"))
    
    //Initialize edges in this DFG
    //connect inputs of node add0
    g.edge("input0", "add0"
      , attrs = Map("operand" -> "0"))
    g.edge("input1", "add0"
      , attrs = Map("operand" -> "1"))
    //connect add0 to output0
    g.edge("add0", "output0"
      , attrs = Map("operand" -> "0"))

  }

val g = new Digraph("Add")
initAdd(g)
//save the .dot file
g.save("Add.dot", ".", print = true)


//visualize
g.render(fileName = "Add.dot", directory = ".", format = "jpg")
viewDFG("./Add.dot.jpg")

## Map this DFG onto the simple block
There are 3 types of mapper:
1. Gurobi ILP mapper (need license)
2. Heuristic search mapper (dynamic programing)
3. OMT mapper (use Z3 SAT solver, not completed)

The following is the usage of ILP mapper:
``` Java
object ILPMap {
  /** Map given DFG(IR) to given MRRG,
   * and write result to file which name is related to filename using FileWriter fw.
   *
   * @example If the file name is "dir/test", this mapper will produce "dir/test_i.txt" (Information TXT)
   *          and "dir/test_r.txt" (Result TXT) when mapping is successful.
   * @param dfg             the given DFG
   * @param mrrg            the given MRRG
   * @param filename        the name we will used to write result
   * @param fw              the FileWriter we used
   * @param separatedPR     a parameter indicating whether ILP placement and routing should be separated
   * @param scheduleControl a parameter indicating whether the latency and skew should be controlled and obtained in ILP
   * @param skewLimit       the limit of skew which only is used when latencyControl is ture
   * @param latencyLimit    the limit of latency which only is used when latencyControl is ture
   * @return the run time of mapper
   */
  def mapping(dfg: DFG, mrrg: MRRG, filename: String = null, fw: FileWriter = null,
              separatedPR: Boolean = false, scheduleControl: Boolean = false,
              skewLimit: Int = 2, latencyLimit: Int = 32): Double = {
        ...
        }
    ...
    }
``` 


In [ ]:
val inputPort = 2
val outputPort = 1
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val block = new SimpleBlockWithReg("Block0")
arch.addBlock(block)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> block / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(block / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

val II = 2
val MRRG = arch.getMRRG(II)


val dfgFilename = "Add.dot"
val dfg = DotReader.loadDot(dfgFilename, II)
val mappingResultFilename = s"Add_ii$II"

object Solver extends Enumeration {
    val Gurobi, Search, Z3Prover = Value
}
val solver = Solver.Search
val separatedPR = true
val scheduleControl = true

var startTime = new Date().getTime()
solver match {
    case Solver.Gurobi => ILPMap.mapping(dfg, MRRG, filename = mappingResultFilename, separatedPR = separatedPR, scheduleControl = scheduleControl, skewLimit = 4, latencyLimit = 15)
    case Solver.Search => SearchMap.mapping(dfg, MRRG, mappingResultFilename, scheduleControl = scheduleControl, skewLimit = 4)
    case Solver.Z3Prover => OmtMap.mapping(dfg, MRRG, filename = mappingResultFilename, separatedPR = separatedPR, scheduleControl = scheduleControl, skewLimit = 4, latencyLimit = 15)
}
var endTime = new Date().getTime()
println("Mapping runtime: " + (endTime - startTime))

var VizGraph = MRRG.GenGraph("Simple_Reg_Mapped")
VizGraph.save("MRRG_simple_Reg_Mapped", ".")
VizGraph.render(fileName = "MRRG_simple_Reg_Mapped", directory = ".", format = "jpg")
viewDFG("./MRRG_simple_Reg_Mapped.jpg")


## Map the Vadd-Reverse DFG onto the 4X4 TileLSUBlock in library

In [ ]:
val rowNum = 4
val colNum = 4
val inputPort = 4
val outputPort = 4
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val tile = new TileLSUBlock("tile_0", colNum, rowNum, inputPort, outputPort,
    useMuxBypass = false, complex = true, isToroid = false, useCounter = true, dataWidth = dataWidth)
arch.addBlock(tile)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> tile / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(tile / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

val II = 1
val MRRG = arch.getMRRG(II)
val dfgFilename = "Vadd_Reverse_.dot"
val dfg = DotReader.loadDot(dfgFilename, II)
val mappingResultFilename = s"ii$II"

object Solver extends Enumeration {
    val Gurobi, Search, Z3Prover = Value
}
val solver = Solver.Search
val separatedPR = true
val scheduleControl = true

var startTime = new Date().getTime()
solver match {
    case Solver.Gurobi => ILPMap.mapping(dfg, MRRG, filename = mappingResultFilename, separatedPR = separatedPR, scheduleControl = scheduleControl, skewLimit = 4, latencyLimit = 15)
    case Solver.Search => SearchMap.mapping(dfg, MRRG, mappingResultFilename, scheduleControl = scheduleControl, skewLimit = 4)
    case Solver.Z3Prover => OmtMap.mapping(dfg, MRRG, filename = mappingResultFilename, separatedPR = separatedPR, scheduleControl = scheduleControl, skewLimit = 4, latencyLimit = 15)
}
var endTime = new Date().getTime()
println("Mapping runtime: " + (endTime - startTime))

var VizGraph = MRRG.GenGraph("MRRG_Mapped")
VizGraph.save("MRRG_Mapped", ".")
VizGraph.render(fileName = "MRRG_Mapped", directory = ".", format = "jpg")
viewDFG("./MRRG_Mapped.jpg")